In [32]:
include("../ode_solver.jl")
include("../numerical_shooting.jl")
include("../examples/example_functions.jl")
include("../visualisation.jl")
include("../finite_difference.jl")
include("../numerical_continuation.jl")

pseudo_arclength (generic function with 1 method)

In [159]:
par_values, conditions = np_continuation(hopf2d, [1 1], 6, "beta", 0:0.01:2, discretisation="shooting")

# Create trace
u1 = scatter(
    x = par_values,
    y = conditions[:,1],
    mode="lines",
    name="u1",
    showlegend=true
    )
u2 = scatter(
    x = par_values,
    y = conditions[:,2],
    mode="lines",
    name="u2",
    showlegend=true
    )

layout = Layout(
    xaxis_title = "parameter",
    # yaxis_type="log",
    yaxis_exponentformat="power",
    # yaxis_title = "",
    width=700, height=350,
    )

plot([u1, u2], layout)

(0.0:0.01:2.0, [-0.0011406615456035674 -9.346260637541235e-14 6.283184897710069; -1.3236927973877773e-15 -1.196504658541496e-18 6.2831849133496345; … ; -1.3236927973877773e-15 -1.196504658541496e-18 6.2831849133496345; -1.3236927973877773e-15 -1.196504658541496e-18 6.2831849133496345])

In [162]:
# include("../numerical_continuation.jl")
np_continuation(hopf2d, [1 1], 6, "bea", 0:0.01:2, discretisation="shooting")

LoadError: UndefVarError: Range not defined

In [181]:
par_values = 0:0.01:2
par_values = 0:0.01:2
# typeof(t)
# Range
# step(par_values)

try 
    step(par_values);
catch 
    error("Please enter a range for the parameter values.")
end

0.01

In [158]:
function pseudo_arclength_eq(secant, v, v_pred)


    estimate = dot(secant, v - v_pred)


    return estimate
end


function pseudo_arclength(f, x0, T, parameter, par_values; discretisation="shooting", arg...)

    """
    Attempts to find a function's solution for each parameter value using the last found solution as an initial guess.
    First solution is found using the inputted initial conditions, x0.

        Parameters:
            f (function): Function which returns a singular value or 1 x n matrix of values.
                The parameter 
            x0 (matrix): Matrix of initial values in the 1 x n form, eg: [1] or [1 1].
            T (float): Initial guess for the period.
            parameter (string): The parameter in the system to vary.
                Allowable method inputs: a, alpha, b, beta, c, d, delta, sigma
            par_values (range): Parameter values to solve between, made with colons, eg: 0:0.1:2.
            discretisation (string): The discretisation to use, either "shooting" or "none."
            arg (list, optional): Arguments to pass to f.

        Returns:
            new_par_values, conditions: the parameter values and corresponding solutions.

        Example Usage:
        pseudo_arclength(hopf2d, [1 1], 6, "beta", 0:0.01:2, discretisation="shooting")
    """

    # convert ints to floats for use in nlsolve
    x0 = [x0 T]
    x0 = [Float64(number) for number in x0] 
    
    # Handle parameter and discretisation assignment
    if discretisation == "shooting"
        if parameter == "a"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, a=par)
        elseif parameter == "alpha"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, alpha=par)
        elseif parameter == "b"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, b=par)
        elseif parameter == "beta"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, beta=par)
        elseif parameter == "c"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, c=par)
        elseif parameter == "d"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, d=par)
        elseif parameter == "delta"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, delta=par)
        elseif parameter == "sigma"
            discretisation = (u0, par) -> shoot(f, u0, phase_index=0, sigma=par)
        end
    elseif discretisation == "none" 
        if parameter == "a"
            discretisation = (u0, par) -> f(u0, a=par)
        elseif parameter == "alpha"
            discretisation = (u0, par) -> f(u0, alpha=par)
        elseif parameter == "b"
            discretisation = (u0, par) -> f(u0, b=par)
        elseif parameter == "beta"
            discretisation = (u0, par) -> f(u0, beta=par)
        elseif parameter == "c"
            discretisation = (u0, par) -> f(u0, c=par)
        elseif parameter == "d"
            discretisation = (u0, par) -> f(u0, d=par)
        elseif parameter == "delta"
            discretisation = (u0, par) -> f(u0, delta=par)
        elseif parameter == "sigma"
            discretisation = (u0, par) -> f(u0, sigma=par)
        end
    else
        error("Invalid discretisation.")
    end

    # Create a list of new parameter values to try
    new_par_values = [par_values[1]; par_values[2]]

    # Check if the parameter difference is positive or negative 
    if par_values[end] - par_values[1] < 0
        end_function = (value) -> value > par_values[end]
    else
        end_function = (value) -> value < par_values[end]
    end

    # Use the first solution as an initial guess for the next
    conditions = nlsolve((u) -> discretisation(u, par_values[1]), x0).zero
    conditions = [conditions; nlsolve((u) -> discretisation(u, new_par_values[2]), conditions).zero]
   
    # Computation
    i = 1
    while end_function(new_par_values[end])

        # Define augmented state vectors
        v0 = [new_par_values[i] conditions[[i],:]]
        v1 = [new_par_values[i+1] conditions[[i+1],:]]
        
        # Find the secant
        secant = v1 - v0
        
        # Find the pseudo-arclength estimate
        v_pred = v1 + secant
        sol = nlsolve((v2) -> [discretisation(v2[:,2:end], v2[1]) pseudo_arclength_eq(secant, v2, v_pred)], v_pred).zero
        
        # Append the new condition and parameter value
        conditions = [conditions; sol[:,2:end]]
        push!(new_par_values, sol[1])

        i += 1
    end
    
    return new_par_values, conditions
end

par_values, conditions = pseudo_arclength(predprey, [1 1], 6, "b", 0.2:0.01:0.3, 
        discretisation="shooting")

([0.2, 0.21, 0.22118486371160345, 0.23379982041904207, 0.24816381001634996, 0.26418464692927585, 0.2777252635564887, 0.29126588018370153, 0.3048064968109144], [0.5778714846989212 0.2861488834424689 20.8168665836801; 0.5469147596381356 0.29310753080924395 20.2249745418589; … ; 0.27015621263841344 0.27015621168428244 16.645615375594517; 0.2701562134773507 0.27015621149547747 16.03318800995114])

In [153]:
# Create trace
u1 = scatter(
    x = par_values,
    y = conditions[:,1],
    mode="lines",
    name="u1",
    showlegend=true
    )
u2 = scatter(
    x = par_values,
    y = conditions[:,2],
    mode="lines",
    name="u2",
    showlegend=true
    )

layout = Layout(
    xaxis_title = "parameter",
    # yaxis_type="log",
    yaxis_exponentformat="power",
    # yaxis_title = "",
    width=700, height=350,
    )

plot([u1, u2], layout)

data: [
  "scatter with fields mode, name, showlegend, type, x, and y",
  "scatter with fields mode, name, showlegend, type, x, and y"
]

layout: "layout with fields height, margin, template, width, xaxis, and yaxis"